In [ ]:
#als nächste: die jahre berücktrichten!!!!!

In [2]:
import os
import pandas as pd
import json
import requests
from meteostat import Daily, Stations
from datetime import datetime
from data_preparation import download_files, files, output_folder
from data_preparation import data_import, data_cleaning, weather_data, geo_data


In [79]:
df=data_import()
df.head()

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.


,Date,Country,City,Specie,count,min,max,median,variance
0,2015-01-06,KR,Jeonju,co,124,0.1,12.3,4.5,55.74
1,2015-01-22,KR,Jeonju,co,116,4.5,10.0,6.7,16.09
2,2015-03-30,KR,Jeonju,co,118,1.2,11.2,5.6,35.98
3,2015-05-27,KR,Jeonju,co,93,2.3,5.6,3.4,6.54
4,2015-02-03,KR,Jeonju,co,133,4.5,13.4,7.8,39.24


In [80]:
def data_cleaning(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

    df["City"] = df["City"].str.lower().str.strip()
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

    df = geo_data(df)

    df = weather_data(df)

    df = convert_date(df)

    df['City'] = df['City'].str.capitalize()

    df = population_data(df)

    #spalten sortieren
    df = df[['Year', 'Month', 'Day', 'Country', 'City', 'Latitude', 'Longitude', 'Population', 'Co', 'No2', 'O3', 'Pm10', 'Pm25',
       'Pressure', 'So2', 'Temperature', 'Wind-gust', 'Wind-speed', 'Dew', 'Humidity', 'Tavg', 'Tmin', 'Tmax', 'Prcp', 'Wdir', 'Wspd', 'Pres',
        ]]

    #Redundante Wetter-Spalten löschen
    df = df.drop(columns=[['Precipitation', 'Pressure', 'Temperature', 'Uvi', 'Wd', 'Wind-gust', 'Wind-speed']], errors='ignore')
    
    #Spalten mit mehr als 90% NaNs löschen
    df = df.loc[:, df.isnull().mean() < 0.9]

    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    return df

In [83]:
df = df.copy()
df = data_cleaning(df)

Keine Geodaten gefunden für: washington d.c.
✅ Wetterdaten gesammelt für 578 Städte
✅ Datei wurde gespeichert: ./data/population_data.csv
✅ Datei wurde gespeichert: ./data/cleaned_data.csv


In [84]:
df.head()

,Year,Month,Day,Country,City,Latitude,Longitude,Population,Co,No2,...,Wind-speed,Dew,Humidity,Tavg,Tmin,Tmax,Prcp,Wdir,Wspd,Pres
0,2014,12,29,AT,Graz,47.06667,15.45000,269997.0,0.1,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,12,29,AT,Innsbruck,47.26266,11.39454,124579.0,0.1,25.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,12,29,AT,Linz,48.30639,14.28611,193814.0,0.1,14.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,12,29,AT,Salzburg,47.79941,13.04399,146631.0,0.1,21.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,12,29,AT,Vienna,48.20849,16.37208,NaN,0.1,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df.dtypes

Year             int64
Month            int32
Day              int32
Country         object
City            object
Latitude       float64
Longitude      float64
Population     float64
Co             float64
No2            float64
O3             float64
Pm10           float64
Pm25           float64
Pressure       float64
So2            float64
Temperature    float64
Wind-gust      float64
Wind-speed     float64
Dew            float64
Humidity       float64
Tavg           float64
Tmin           float64
Tmax           float64
Prcp           float64
Wdir           float64
Wspd           float64
Pres           float64
dtype: object

In [ ]:
# alle raus, wo mehr al 90% der daten fehlen (aqi, D, Mepaqi, Neph, Pm1, Pol, Precipittion, psi, Temperature	Uvi	Wd	Wind-gust	Wind-speed ) # type: ignore
# wetterdaten aus einer quelle
# spalten sortieren
# wo sind die pop daten

In [71]:
# # Funktion für Übersicht über dtypes, missing values, unique values etc.
def overview(df):
    '''
     Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
     VARs
         df: Der zu betrachtende DataFrame
     RETURNS:
         None
     '''
    display(pd.DataFrame({'dtype': df.dtypes,
                           'total': df.count(),
                           'missing': df.isna().sum(),
                           'missing%': df.isna().mean()*100,
                           'n_uniques': df.nunique(),
                           'uniques%': df.nunique()/df.shape[0]*100,
                           'uniques': [df[col].unique() for col in df.columns]
                          }))
    #return overview(df)


In [72]:
overview(df)

,dtype,total,missing,missing%,n_uniques,uniques%,uniques
Country,object,1693987,0,0.000000,95,0.005608,"[AT, AU, BE, BO, BR, CA, CH, CL, CN, CO, CY, C..."
City,object,1693987,0,0.000000,616,0.036364,"[Graz, Innsbruck, Linz, Salzburg, Vienna, Bris..."
Aqi,float64,39782,1654205,97.651576,329,0.019422,"[nan, 32.0, 35.0, 29.0, 21.0, 40.0, 28.0, 31.0..."
Co,float64,1062683,631304,37.267346,1012,0.059741,"[0.1, 1.9, 2.1, nan, 2.3, 2.0, 3.4, 5.5, 3.8, ..."
D,float64,2083,1691904,99.877036,67,0.003955,"[nan, 0.0, 9.0, 10.0, 11.0, 8.0, 15.0, 18.0, 1..."
Dew,float64,999185,694802,41.015781,661,0.039020,"[nan, 16.0, 14.0, 0.5, 2.5, 1.0, 11.0, 18.0, 2..."
Humidity,float64,1305907,388080,22.909267,1131,0.066766,"[nan, 0.0, 100.0, 99.0, 94.0, 84.0, 82.0, 66.0..."
Mepaqi,float64,2778,1691209,99.836008,809,0.047757,"[nan, 81.0, 81.3, 48.2, 115.8, 59.5, 62.9, 62...."
Neph,float64,6730,1687257,99.602712,65,0.003837,"[nan, 5.0, 7.0, 14.0, 13.0, 15.0, 19.0, 20.0, ..."
No2,float64,1407779,286208,16.895525,1302,0.076860,"[9.0, 25.6, 14.2, 21.1, 4.6, 0.7, nan, 6.5, 4...."


In [75]:
df.columns

Index(['Country', 'City', 'Co', 'Dew', 'Humidity', 'No2', 'O3', 'Pm10', 'Pm25',
       'Pressure', 'So2', 'Temperature', 'Wind-gust', 'Wind-speed', 'Latitude',
       'Longitude', 'Tavg', 'Tmin', 'Tmax', 'Prcp', 'Wdir', 'Wspd', 'Pres',
       'Year', 'Month', 'Day', 'Population'],
      dtype='object')

In [74]:
df = df.loc[:, df.isnull().mean() < 0.9]
df.head()

,Country,City,Co,Dew,Humidity,No2,O3,Pm10,Pm25,Pressure,...,Tmin,Tmax,Prcp,Wdir,Wspd,Pres,Year,Month,Day,Population
0,AT,Graz,0.1,NaN,NaN,9.0,NaN,13.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,269997.0
1,AT,Innsbruck,0.1,NaN,NaN,25.6,NaN,25.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,124579.0
2,AT,Linz,0.1,NaN,NaN,14.2,NaN,25.0,74.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,193814.0
3,AT,Salzburg,0.1,NaN,NaN,21.1,NaN,21.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,146631.0
4,AT,Vienna,0.1,NaN,NaN,9.0,NaN,20.0,65.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN


In [40]:
def convert_date(df):
    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Split 'Date' column into 'year', 'month' and 'day'
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day

    # Remove 'Date' column
    if 'Date' in df.columns:
        df.drop(columns=['Date'], inplace=True)
    
    return (df)

In [ ]:
df_=convert_date(df)
df_.head()

,Country,City,aqi,co,d,dew,humidity,mepaqi,neph,no2,...,tavg,tmin,tmax,prcp,wdir,wspd,pres,year,month,day
0,AT,graz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29
1,AT,innsbruck,NaN,0.1,NaN,NaN,NaN,NaN,NaN,25.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29
2,AT,linz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,14.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29
3,AT,salzburg,NaN,0.1,NaN,NaN,NaN,NaN,NaN,21.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29
4,AT,vienna,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29


In [ ]:
def population_data(df):
    '''
    Fügt dem jeder Stadt Einwohner hinzu
    '''
    df = df.copy()
    
    df_population = pd.read_csv(
                    './data/population.csv',
                    sep=',', 
                    header=0,
                    engine='python',  # Hilft oft bei Parsing-Problemen
                    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
                    encoding='utf-8')  # Falls Sonderzeichen vorhanden sind

    df_population = df_population[['City', 'Value', 'Year']]

    df_population.rename(columns={'Value': 'Population'}, inplace=True)

    df_population.dropna(inplace=True)
    df_population['Population'] = df_population['Population'].astype(int)

    # Merge der beiden DataFrames basierend auf der "City"-Spalte
    df.columns = df.columns.str.capitalize()
    df['Year'] = df['Year'].astype(int)
    df_population['Year'] = df_population['Year'].astype(int)
    df = pd.merge(df, df_population, on=['City', 'Year'], how='left')
    

    output_path = './data/population_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df_population.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    return(df)

In [63]:
df_['City'] = df_['City'].str.capitalize()
df_.head()

,Country,City,Aqi,Co,D,Dew,Humidity,Mepaqi,Neph,No2,...,Tmax,Prcp,Wdir,Wspd,Pres,Year,Month,Day,Population_x,Population_y
0,AT,Graz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
1,AT,Innsbruck,NaN,0.1,NaN,NaN,NaN,NaN,NaN,25.6,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
2,AT,Linz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,14.2,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
3,AT,Salzburg,NaN,0.1,NaN,NaN,NaN,NaN,NaN,21.1,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
4,AT,Vienna,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN


In [44]:
df_= population_data(df_)
print(df_.shape)
df_.head()

✅ Datei wurde gespeichert: ./data/population_data.csv
(1568540, 38)


,Country,City,Aqi,Co,D,Dew,Humidity,Mepaqi,Neph,No2,...,Tmax,Prcp,Wdir,Wspd,Pres,Year,Month,Day,Population_x,Population_y
0,AT,graz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
1,AT,innsbruck,NaN,0.1,NaN,NaN,NaN,NaN,NaN,25.6,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
2,AT,linz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,14.2,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
3,AT,salzburg,NaN,0.1,NaN,NaN,NaN,NaN,NaN,21.1,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN
4,AT,vienna,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN,NaN


In [31]:
df_population = population_data(df)
df_population.head()

✅ Datei wurde gespeichert: ./data/population_data.csv


,City,Population,Year
0,MARIEHAMN,11785,2023
1,MARIEHAMN,11750,2022
2,MARIEHAMN,11724,2021
3,MARIEHAMN,11692,2020
4,MARIEHAMN,11711,2019


In [32]:
df.columns = df.columns.str.capitalize()
df['Year'] = df['Year'].astype(int)
df_population['Year'] = df_population['Year'].astype(int)
df = pd.merge(df, df_population, on=['City', 'Year'], how='left')


In [33]:
df.head()

,Country,City,Aqi,Co,D,Dew,Humidity,Mepaqi,Neph,No2,...,Tmin,Tmax,Prcp,Wdir,Wspd,Pres,Year,Month,Day,Population
0,AT,graz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN
1,AT,innsbruck,NaN,0.1,NaN,NaN,NaN,NaN,NaN,25.6,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN
2,AT,linz,NaN,0.1,NaN,NaN,NaN,NaN,NaN,14.2,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN
3,AT,salzburg,NaN,0.1,NaN,NaN,NaN,NaN,NaN,21.1,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN
4,AT,vienna,NaN,0.1,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,12,29,NaN


In [ ]:
def convert_date(df):
    """
    Teilt die Spalte Date in Year, Month, Day auf
    """
    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Split 'Date' column into 'year', 'month' and 'day'
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day

    # Remove 'Date' column
    if 'Date' in df.columns:
        df.drop(columns=['Date'], inplace=True)

In [ ]:
df=data_cleaning(df)
df.head()

In [ ]:
df_population = pd.read_csv(
    '/Users/whypk/01Projekte/air_quality/data/population.csv',
    sep=',', 
    header=0,
    engine='python',  # Hilft oft bei Parsing-Problemen
    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
    encoding='utf-8'  # Falls Sonderzeichen vorhanden sind
)

In [ ]:
print(df_population.head())
print(df_population.columns)
print(df_population.dtypes)
print(df_population.shape)

In [ ]:
df_population= df_population[['City', 'Value', 'Year']]
df_population.head()


In [ ]:

#df_population['population'] = df_population['population'].astype(int)
#df_population["City"] = df_population["City"].str.lower().str.strip()
df_population.rename(columns={'Value': 'Population'}, inplace=True)
df_population.head()

In [ ]:
df_population.dropna(inplace=True)
df_population['Population'] = df_population['Population'].astype(int)
df_population.head()

In [ ]:
df_ = pd.merge(df, df_population, on=['City', 'Year'], how='left')

In [ ]:
output_path = './data/population_data.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_population.to_csv(output_path, index=False)
print(f"✅ Datei wurde gespeichert: {output_path}")

In [ ]:
# Merge der beiden DataFrames basierend auf der "City"-Spalte
df_merged = pd.merge(df, df_population, on='City', how='left')

In [ ]:
# Funktion für Übersicht über dtypes, missing values, unique values etc.
def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))
overview(df_population)

In [ ]:
df_filtered_population = df_population[df_population['City'].isin(df['City'])][['Country or Area','City', 'Year', 'City type', 'Value']]
df_filtered_population['Value'] = df_filtered_population['Value'].astype(int)
df_filtered_population.head()